<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/Geeta_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7
!pip install -U langchain-community
!pip -qqq install langchain-groq==0.1.3 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 111.

In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [5]:
import os
import re
import requests
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

def format_response (response: str) -> str:
    entries = re.split(r" (?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]

os.environ ["GROQ_API_KEY"] = "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"

In [6]:
mistral_llm = ChatGroq(temperature = 0.2,model_name="llama3-70b-8192")

In [8]:
import pandas as pd
from bs4 import BeautifulSoup
from html2text import HTML2Text

# Replace 'your_file.csv' with the actual path to your CSV file
csv_file_path = '/content/modified_meaning.csv'

# Replace 'your_column_name' with the name of the column you want to extract
column_name = 'meaning'

# Read the first 500 rows of the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, nrows=600)

# Create a list to store transformed content
docs_transformed = []

# Create an HTML2Text object
html2text = HTML2Text()

# Extract data from the specified column into a list
for index, row in df.iterrows():
    html_content = row[column_name]
    html_content = str(html_content)
    soup = BeautifulSoup(html_content, 'html.parser')
    plain_text = html2text.handle(str(soup))
    docs_transformed.append(plain_text)

class PageContentWrapper:
    def __init__(self, page_content, metadata={}):
        self.page_content = page_content
        self.metadata = metadata

# Assuming plain_text is the content you want to chunk
docs_transformed_wrapped = [PageContentWrapper(content) for content in docs_transformed]

# Now use docs_transformed_wrapped with CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(docs_transformed_wrapped)



In [9]:
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Create prompt template
prompt_template = """
note while returning final answer please print little bit of context from docs that you have used to generate answer
### [INST] Instruction: Answer the question based on your docs knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)


rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)
result = rag_chain.invoke("i secured low marks in my exams what should i do")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
result['context']

[Document(page_content='Purport In the Śrīmad-Bhāgavatam (1.5.17) Śrī Nārada Muni instructs Vyāsadeva\nas follows: tyaktvā sva-dharmaṁ caraṇāmbujaṁ harerbhajann apakvo ’tha patet\ntato yadiyatra kva vābhadram abhūd amuṣya kiṁko vārtha āpto ’bhajatāṁ sva-\ndharmataḥ “If someone gives up all material prospects and takes complete\nshelter of the Supreme Personality of Godhead, there is no loss or degradation\nin any way. On the other hand a nondevotee may fully engage in his\noccupational duties and yet not gain anything.” For material prospects there\nare many activities, both scriptural and customary. A transcendentalist is\nsupposed to give up all material activities for the sake of spiritual\nadvancement in life, Kṛṣṇa consciousness. One may argue that by Kṛṣṇa\nconsciousness one may attain the highest perfection if it is completed, but if\none does not attain such a perfectional stage, then he loses both materially\nand spiritually. It is enjoined in the scriptures that one has to su

In [12]:
text = result['text']

# Add line breaks to format the text as a paragraph
formatted_text = text.replace('\n', ' ')  # Replace existing line breaks with spaces
formatted_text = formatted_text.replace('. ', '.\n\n')  # Add double line breaks after periods

# Print the formatted text
print(formatted_text)

Don't worry, my friend! It's completely normal to face setbacks in life, including not doing well in exams.

Here's some advice from the Bhagavatam:  "...even if one does not attain such a perfectional stage, then he loses both materially and spiritually.

It is enjoined in the scriptures that one has to suffer the reaction for not executing prescribed duties; therefore one who fails to discharge transcendental activities properly becomes subjected to these reactions.

The Bhāgavatam assures the unsuccessful transcendentalist that there need be no worries."  In other words, don't worry about the outcome; instead, focus on the process.

Reflect on what went wrong and how you can improve.

Take this as an opportunity to learn and grow.

Remember, it's not about the marks you scored, but about the effort you put in and the lessons you learn from the experience.

 As the Bhagavatam says, "One may argue that by Kṛṣṇa consciousness one may attain the highest perfection if it is completed, bu